In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# function to find the key of the value when label encoding the classes
def key_find(classes, pred):
    for key, val in classes.items():
        if pred == val:
            return key
        
        
label_encoder = LabelEncoder()

# get data and instantiate Random Forest Classifier
X = pd.read_csv("/Users/kritinshanmugam/Desktop/gene_expression.csv")
y = pd.read_csv("/Users/kritinshanmugam/Desktop/label.csv")

classifier = RandomForestClassifier(random_state=31)

# Handle missing values
X = X.dropna()
y = y.dropna()

# Drop ID column
X.drop(columns=X.columns[0], axis=1)

# Standardize features
X = (X - X.mean()) / X.std()



In [ ]:
from sklearn.model_selection import train_test_split

y.drop(columns=y.columns[0], axis=1)

y_orig = y.copy()

# One-hot encode the target variable
y = pd.DataFrame(label_encoder.fit_transform(y))

# Print the mapping between original classes and encoded integers
class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Split the data into train and test sets

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

x_train = X.iloc[500:, :]
x_test = X.iloc[0:500, :]

y_train = y.iloc[500:]
y_test = y.iloc[0:500]

In [ ]:
classifier.fit(x_train, y_train)
y_predictions = classifier.predict(x_test)

predicted = pd.DataFrame({'predicted_class': []})


for prediction in y_predictions:
    class_pred = key_find(class_mapping, prediction)
    predicted.loc[len(predicted)] = class_pred

incorrect_list = []

for row in range(len(predicted)):
    if predicted.iloc[row, 0] != y_orig.iloc[row, 0]:
        incorrect_list.append(row)
        


In [ ]:
import shap

explaine = shap.TreeExplainer(model)
shap_values = explaine.shap_values(X)
row = 1
shap.waterfall_plot(shap.Explanation(values=shap_values[0][row],
                                     base_values=explaine.expected_value[0], data=x_test.iloc[row],
                                     feature_names=x_test.columns.tolist()))

shap.plots.waterfall(shap_values[0][row])